In [ ]:
from torchvision import transforms
from PIL import Image 
import numpy as np
import os
import json
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split

from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
class DataProcessor:
    def __init__(self, folder_path, disease_list):
        self.folder_path = folder_path
        self.disease_list = disease_list
        self.folders_with_diseases_labels = {}
        self.folder_name_with_diseases = []
        self.label_counts = None

    def read_data(self):
        for root, dirs, files in os.walk(os.path.join(self.folder_path, 'imgs')):
            for folder_name in dirs:
                folder_path = os.path.join(root, folder_name)
                
                detection_file_path = os.path.join(folder_path, 'detection.json')
                with open(detection_file_path, 'r') as detection_file:
                    detection_data = json.load(detection_file)

                    disease_labels = [label for item in detection_data for label in item.keys() if label in self.disease_list]

                    # merge labels for images with multiple labels
                    if disease_labels:
                        merged_label = '-'.join(sorted(set(disease_labels)))
                        self.folders_with_diseases_labels[folder_name] = merged_label
                        self.folder_name_with_diseases.append(folder_name)

    def delete_folders(self):
        # frequency of each merged label
        self.label_counts = Counter(self.folders_with_diseases_labels.values())

        # delete folders with label counts <= 3
        folders_to_delete = [folder_name for folder_name, label in self.folders_with_diseases_labels.items() if self.label_counts[label] <= 3]

        for folder_name in folders_to_delete:
            del self.folders_with_diseases_labels[folder_name]
            self.folder_name_with_diseases.remove(folder_name)
            
    def get_training_data(self):
        training_data = []
        for folder_name, label in self.folders_with_diseases_labels.items():
            folder_path = os.path.join(self.folder_path, 'imgs', folder_name)
            image_path = os.path.join(folder_path, 'source.jpg')
            training_data.append((image_path, label))
        return training_data


folder_path = 'Slake1.0'
disease_list = ['Mass', 'Pneumothorax', 'Brain Enhancing Tumor', 'Pneumonia', 'Nodule', 'Atelectasis',
               'Liver Cancer', 'Effusion','Cardiomegaly', 'Brain Edema', 'Lung Cancer', 'Infiltrate', 
                'Brain Non-enhancing Tumor']
data_processor = DataProcessor(folder_path, disease_list)
data_processor.read_data()
data_processor.delete_folders()
#training_data = data_processor.get_training_data()


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_processor, folder_names, transform=None, is_train=True):
        self.data_processor = data_processor
        self.folder_names = folder_names
        self.transform = transform
        self.is_train = is_train

        # map labels to index
        self.label_to_index = {label: idx for idx, label in enumerate(set(data_processor.folders_with_diseases_labels.values()))}

    def __len__(self):
        return len(self.folder_names)

    def __getitem__(self, idx):
        folder_name = self.folder_names[idx]
        folder_path = os.path.join(self.data_processor.folder_path, 'imgs', folder_name)
        
        # read images 'source.jpg' in each folder
        image_path = os.path.join(folder_path, 'source.jpg')
        image = Image.open(image_path).convert('RGB')
        
        if self.is_train:
            # Random horizontal flip
            if np.random.rand() > 0.5:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)

            # Random vertical flip
            if np.random.rand() > 0.5:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)

            # Random rotation (up to 30 degrees)
            angle = np.random.uniform(-30, 30)
            image = image.rotate(angle)


            # Random crop
            #i, j, h, w = transforms.RandomCrop.get_params(image, output_size=(256, 256))
            #image = transforms.functional.crop(image, i, j, h, w)

        if self.transform:
            image = self.transform(image)

        label = self.data_processor.folders_with_diseases_labels[folder_name]
        label = self.label_to_index[label]

        # converting images to tensor
        if not torch.is_tensor(image):
            image = transforms.ToTensor()(image)

        return image, label


In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 70% training and 30% validation sets
training_data, validation_data = train_test_split(data_processor.folder_name_with_diseases, test_size=0.2, random_state=42, shuffle = True)

# CustomDataset for both training and validation
train_dataset = CustomDataset(data_processor, folder_names=training_data, transform=transform, is_train=True)
val_dataset = CustomDataset(data_processor, folder_names=validation_data, transform=transform, is_train=False)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=1)

In [ ]:
class ImageClassificationModel(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.4, l2_regularization=1e-3):
        super(ImageClassificationModel, self).__init__()

        vgg16 = models.vgg16(pretrained=True)
        self.features = vgg16.features

        # reduce spatial dimensions
        self.global_avg_pooling = nn.AdaptiveAvgPool2d(1)

        #classification part
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.global_avg_pooling(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes=14
model = ImageClassificationModel(num_classes=num_classes).to(device)

label_counts = {
    'Atelectasis': 25,
    'Brain Edema-Brain Enhancing Tumor-Brain Non-enhancing Tumor': 73,
    'Nodule': 26,
    'Cardiomegaly': 33,
    'Mass': 20,
    'Pneumonia': 28,
    'Brain Edema-Brain Non-enhancing Tumor': 21,
    'Pneumothorax': 14,
    'Lung Cancer': 18,
    'Effusion': 12,
    'Brain Edema': 22,
    'Brain Edema-Brain Enhancing Tumor': 12,
    'Infiltrate': 9,
    'Liver Cancer': 30
    }

# inverse class frequencies
total_samples = sum(label_counts.values())
class_weights = {label: total_samples / (len(label_counts) * count) for label, count in label_counts.items()}

# to normalize the weights
total_weights = sum(class_weights.values())
class_weights = {label: weight / total_weights for label, weight in class_weights.items()}

# converting to tensor
class_weights_tensor = torch.tensor(list(class_weights.values())).to(device)

# weighted CrossEntropyLoss to handle imbalance classes
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
#criterion = nn.CrossEntropyLoss()

weight_decay = 5*1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=weight_decay)
#optimizer = optim.RMSprop(model.parameters(), lr=1e-3, weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=weight_decay)

def calculate_accuracy(outputs, targets):
    _, predicted = torch.max(outputs, 1)
    return accuracy_score(targets.cpu().numpy(), predicted.cpu().numpy())

def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    total_samples = 0
    total_correct = 0
    
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)

        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()

    epoch_loss = total_loss / total_samples
    epoch_accuracy = total_correct / total_samples

    return epoch_loss, epoch_accuracy

def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    total_correct = 0
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)

            loss = criterion(outputs, targets)

            total_loss += loss.item() * inputs.size(0)
            
            total_samples += inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == targets).sum().item()

    epoch_loss = total_loss / total_samples
    epoch_accuracy = total_correct / total_samples

    return epoch_loss, epoch_accuracy

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
num_epochs = 50
for epoch in range(num_epochs):
    
    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion)
    #scheduler.step()
    val_loss, val_accuracy = validate(model, val_loader, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    


In [ ]:
# training and validation loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()

# to save files
save_path = '/storage/homefs/zh21i037/'
filename = 'losses and accuracies.png'
save_filename = os.path.join(save_path, filename)

plt.savefig(save_filename)
plt.close()
